In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.9 MB/s eta 0:00:0000:0100:01


In [3]:
#!pip install -U weaviate-client
!pip install "weaviate-client==3.*" # Version 4 of the client does not work

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 7.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import os

import json
import re

In [5]:
WCS_ENDPOINT = "https://my-sandbox1-1486fdzz.weaviate.network/" # Weaviate
WCS_API_KEY = "YOUR-API-KEY" # Weaviate
OPENAI_API_KEY = 'YOUR-API-KEY'
COHERE_API_KEY = 'YOUR-API-KEY'

VECTOR_DB_NAME = 'ARXIV_100SAMPLE_VDB'
BATCH_SIZE = 100

SAMPLE_SIZE = 500

In [6]:
# You can access each paper directly on ArXiv using these links:
# https://arxiv.org/abs/{id}: Page for this paper including its abstract and further links
# https://arxiv.org/pdf/{id}: Direct link to download the PDF

In [7]:
# https://arxiv.org/abs/0704.1020

In [8]:
os.listdir('../input/arxiv')

['arxiv-metadata-oai-snapshot.json']

In [9]:
# All Arxiv category codes

# https://www.kaggle.com/code/artgor/arxiv-metadata-exploration

# https://arxiv.org/help/api/user-manual
category_map = {'astro-ph': 'Astrophysics',
'astro-ph.CO': 'Cosmology and Nongalactic Astrophysics',
'astro-ph.EP': 'Earth and Planetary Astrophysics',
'astro-ph.GA': 'Astrophysics of Galaxies',
'astro-ph.HE': 'High Energy Astrophysical Phenomena',
'astro-ph.IM': 'Instrumentation and Methods for Astrophysics',
'astro-ph.SR': 'Solar and Stellar Astrophysics',
'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
'cond-mat.mes-hall': 'Mesoscale and Nanoscale Physics',
'cond-mat.mtrl-sci': 'Materials Science',
'cond-mat.other': 'Other Condensed Matter',
'cond-mat.quant-gas': 'Quantum Gases',
'cond-mat.soft': 'Soft Condensed Matter',
'cond-mat.stat-mech': 'Statistical Mechanics',
'cond-mat.str-el': 'Strongly Correlated Electrons',
'cond-mat.supr-con': 'Superconductivity',
'cs.AI': 'Artificial Intelligence',
'cs.AR': 'Hardware Architecture',
'cs.CC': 'Computational Complexity',
'cs.CE': 'Computational Engineering, Finance, and Science',
'cs.CG': 'Computational Geometry',
'cs.CL': 'Computation and Language',
'cs.CR': 'Cryptography and Security',
'cs.CV': 'Computer Vision and Pattern Recognition',
'cs.CY': 'Computers and Society',
'cs.DB': 'Databases',
'cs.DC': 'Distributed, Parallel, and Cluster Computing',
'cs.DL': 'Digital Libraries',
'cs.DM': 'Discrete Mathematics',
'cs.DS': 'Data Structures and Algorithms',
'cs.ET': 'Emerging Technologies',
'cs.FL': 'Formal Languages and Automata Theory',
'cs.GL': 'General Literature',
'cs.GR': 'Graphics',
'cs.GT': 'Computer Science and Game Theory',
'cs.HC': 'Human-Computer Interaction',
'cs.IR': 'Information Retrieval',
'cs.IT': 'Information Theory',
'cs.LG': 'Machine Learning',
'cs.LO': 'Logic in Computer Science',
'cs.MA': 'Multiagent Systems',
'cs.MM': 'Multimedia',
'cs.MS': 'Mathematical Software',
'cs.NA': 'Numerical Analysis',
'cs.NE': 'Neural and Evolutionary Computing',
'cs.NI': 'Networking and Internet Architecture',
'cs.OH': 'Other Computer Science',
'cs.OS': 'Operating Systems',
'cs.PF': 'Performance',
'cs.PL': 'Programming Languages',
'cs.RO': 'Robotics',
'cs.SC': 'Symbolic Computation',
'cs.SD': 'Sound',
'cs.SE': 'Software Engineering',
'cs.SI': 'Social and Information Networks',
'cs.SY': 'Systems and Control',
'econ.EM': 'Econometrics',
'eess.AS': 'Audio and Speech Processing',
'eess.IV': 'Image and Video Processing',
'eess.SP': 'Signal Processing',
'gr-qc': 'General Relativity and Quantum Cosmology',
'hep-ex': 'High Energy Physics - Experiment',
'hep-lat': 'High Energy Physics - Lattice',
'hep-ph': 'High Energy Physics - Phenomenology',
'hep-th': 'High Energy Physics - Theory',
'math.AC': 'Commutative Algebra',
'math.AG': 'Algebraic Geometry',
'math.AP': 'Analysis of PDEs',
'math.AT': 'Algebraic Topology',
'math.CA': 'Classical Analysis and ODEs',
'math.CO': 'Combinatorics',
'math.CT': 'Category Theory',
'math.CV': 'Complex Variables',
'math.DG': 'Differential Geometry',
'math.DS': 'Dynamical Systems',
'math.FA': 'Functional Analysis',
'math.GM': 'General Mathematics',
'math.GN': 'General Topology',
'math.GR': 'Group Theory',
'math.GT': 'Geometric Topology',
'math.HO': 'History and Overview',
'math.IT': 'Information Theory',
'math.KT': 'K-Theory and Homology',
'math.LO': 'Logic',
'math.MG': 'Metric Geometry',
'math.MP': 'Mathematical Physics',
'math.NA': 'Numerical Analysis',
'math.NT': 'Number Theory',
'math.OA': 'Operator Algebras',
'math.OC': 'Optimization and Control',
'math.PR': 'Probability',
'math.QA': 'Quantum Algebra',
'math.RA': 'Rings and Algebras',
'math.RT': 'Representation Theory',
'math.SG': 'Symplectic Geometry',
'math.SP': 'Spectral Theory',
'math.ST': 'Statistics Theory',
'math-ph': 'Mathematical Physics',
'nlin.AO': 'Adaptation and Self-Organizing Systems',
'nlin.CD': 'Chaotic Dynamics',
'nlin.CG': 'Cellular Automata and Lattice Gases',
'nlin.PS': 'Pattern Formation and Solitons',
'nlin.SI': 'Exactly Solvable and Integrable Systems',
'nucl-ex': 'Nuclear Experiment',
'nucl-th': 'Nuclear Theory',
'physics.acc-ph': 'Accelerator Physics',
'physics.ao-ph': 'Atmospheric and Oceanic Physics',
'physics.app-ph': 'Applied Physics',
'physics.atm-clus': 'Atomic and Molecular Clusters',
'physics.atom-ph': 'Atomic Physics',
'physics.bio-ph': 'Biological Physics',
'physics.chem-ph': 'Chemical Physics',
'physics.class-ph': 'Classical Physics',
'physics.comp-ph': 'Computational Physics',
'physics.data-an': 'Data Analysis, Statistics and Probability',
'physics.ed-ph': 'Physics Education',
'physics.flu-dyn': 'Fluid Dynamics',
'physics.gen-ph': 'General Physics',
'physics.geo-ph': 'Geophysics',
'physics.hist-ph': 'History and Philosophy of Physics',
'physics.ins-det': 'Instrumentation and Detectors',
'physics.med-ph': 'Medical Physics',
'physics.optics': 'Optics',
'physics.plasm-ph': 'Plasma Physics',
'physics.pop-ph': 'Popular Physics',
'physics.soc-ph': 'Physics and Society',
'physics.space-ph': 'Space Physics',
'q-bio.BM': 'Biomolecules',
'q-bio.CB': 'Cell Behavior',
'q-bio.GN': 'Genomics',
'q-bio.MN': 'Molecular Networks',
'q-bio.NC': 'Neurons and Cognition',
'q-bio.OT': 'Other Quantitative Biology',
'q-bio.PE': 'Populations and Evolution',
'q-bio.QM': 'Quantitative Methods',
'q-bio.SC': 'Subcellular Processes',
'q-bio.TO': 'Tissues and Organs',
'q-fin.CP': 'Computational Finance',
'q-fin.EC': 'Economics',
'q-fin.GN': 'General Finance',
'q-fin.MF': 'Mathematical Finance',
'q-fin.PM': 'Portfolio Management',
'q-fin.PR': 'Pricing of Securities',
'q-fin.RM': 'Risk Management',
'q-fin.ST': 'Statistical Finance',
'q-fin.TR': 'Trading and Market Microstructure',
'quant-ph': 'Quantum Physics',
'stat.AP': 'Applications',
'stat.CO': 'Computation',
'stat.ME': 'Methodology',
'stat.ML': 'Machine Learning',
'stat.OT': 'Other Statistics',
'stat.TH': 'Statistics Theory'}


In [10]:
# Choose category codes

selected_cats_dict = {
        'cs.AI': 'Artificial Intelligence',
        'cs.CV': 'Computer Vision and Pattern Recognition',
        'cs.ET': 'Emerging Technologies',
        'cs.MA': 'Multiagent Systems',
        'cs.LG': 'Machine Learning',
        'cs.RO': 'Robotics',
        'eess.AS': 'Audio and Speech Processing',
        'eess.IV': 'Image and Video Processing',
        'eess.SP': 'Signal Processing',
        'stat.ML': 'Machine Learning',
        }

selected_cats_list = list(selected_cats_dict.keys())

selected_cats_list

['cs.AI',
 'cs.CV',
 'cs.ET',
 'cs.MA',
 'cs.LG',
 'cs.RO',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'stat.ML']

## Load the Arxiv metadata

In [11]:
# https://www.kaggle.com/code/matthewmaddock/nlp-arxiv-dataset-transformers-and-umap

# Load all the available metadata

cols = ['id', 'title', 'abstract', 'categories']
data = []
file_name = '/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json'

with open(file_name, encoding='latin-1') as f:
    for line in f:
        doc = json.loads(line)
        lst = [doc['id'], doc['title'], doc['abstract'], doc['categories']]
        data.append(lst)

df = pd.DataFrame(data=data, columns=cols)

df.head()

,id,title,abstract,categories
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,hep-ph
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",math.CO cs.CG
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,physics.gen-ph
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,math.CO
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,math.CA math.FA


In [12]:
df.shape

(2417693, 4)

In [21]:
# Put the cat codes into a list

def get_cat_codes(x):
    
    cat_list = x.split(' ')
        
    return cat_list

df['code_list'] = df['categories'].apply(get_cat_codes)

print(df.shape)

df.head()

(2417693, 6)


,id,title,abstract,categories,code_list,selected_cats
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,hep-ph,[hep-ph],no
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",math.CO cs.CG,"[math.CO, cs.CG]",no
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,physics.gen-ph,[physics.gen-ph],no
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,math.CO,[math.CO],no
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,math.CA math.FA,"[math.CA, math.FA]",no


In [22]:
# Filter out onlt the codes we selected

def filter_by_code(x_list):
    
    for item in x_list:
        if item in selected_cats_list:
            
            return 'yes'
        else:
            return 'no'
        
df['selected_cats'] = df['code_list'].apply(filter_by_code)

df.head()

,id,title,abstract,categories,code_list,selected_cats
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,hep-ph,[hep-ph],no
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",math.CO cs.CG,"[math.CO, cs.CG]",no
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,physics.gen-ph,[physics.gen-ph],no
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,math.CO,[math.CO],no
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,math.CA math.FA,"[math.CA, math.FA]",no


In [23]:
df_filtered = df[df['selected_cats'] == 'yes']

df_filtered = df_filtered.reset_index(drop=True)

print(df_filtered.shape)

df_filtered.head()

(259463, 6)


,id,title,abstract,categories,code_list,selected_cats
0,0704.1020,The on-line shortest path problem under partia...,The on-line shortest path problem is conside...,cs.LG cs.SC,"[cs.LG, cs.SC]",yes
1,0704.1028,A neural network approach to ordinal regression,Ordinal regression is an important type of l...,cs.LG cs.AI cs.NE,"[cs.LG, cs.AI, cs.NE]",yes
2,0704.1267,Text Line Segmentation of Historical Documents...,There is a huge amount of historical documen...,cs.CV,[cs.CV],yes
3,0704.1274,Parametric Learning and Monte Carlo Optimization,This paper uncovers and explores the close r...,cs.LG,[cs.LG],yes
4,0704.1394,Calculating Valid Domains for BDD-Based Intera...,In these notes we formally describe the func...,cs.AI,[cs.AI],yes


In [24]:
i = 0

print(df_filtered.loc[i, 'id'])
print(df_filtered.loc[i, 'title'])
print(df_filtered.loc[i, 'abstract'])

0704.1020
The on-line shortest path problem under partial monitoring
  The on-line shortest path problem is considered under various models of
partial monitoring. Given a weighted directed acyclic graph whose edge weights
can change in an arbitrary (adversarial) way, a decision maker has to choose in
each round of a game a path between two distinguished vertices such that the
loss of the chosen path (defined as the sum of the weights of its composing
edges) be as small as possible. In a setting generalizing the multi-armed
bandit problem, after choosing a path, the decision maker learns only the
weights of those edges that belong to the chosen path. For this problem, an
algorithm is given whose average cumulative loss in n rounds exceeds that of
the best path, matched off-line to the entire sequence of the edge weights, by
a quantity that is proportional to 1/\sqrt{n} and depends only polynomially on
the number of edges of the graph. The algorithm can be implemented with linear
complex

## Clean the text

In [25]:
# Replace newline characters ('\n') with a space
# Remove leading and trailing spaces

def clean_text(x):
    
    # Replace newline characters with a space
    new_text = x.replace("\n", " ")
    # Remove leading and trailing spaces
    new_text = new_text.strip()
    
    return new_text

df_filtered['title'] = df_filtered['title'].apply(clean_text)
df_filtered['abstract'] = df_filtered['abstract'].apply(clean_text)

df_filtered.head()

,id,title,abstract,categories,code_list,selected_cats
0,0704.1020,The on-line shortest path problem under partia...,The on-line shortest path problem is considere...,cs.LG cs.SC,"[cs.LG, cs.SC]",yes
1,0704.1028,A neural network approach to ordinal regression,Ordinal regression is an important type of lea...,cs.LG cs.AI cs.NE,"[cs.LG, cs.AI, cs.NE]",yes
2,0704.1267,Text Line Segmentation of Historical Documents...,There is a huge amount of historical documents...,cs.CV,[cs.CV],yes
3,0704.1274,Parametric Learning and Monte Carlo Optimization,This paper uncovers and explores the close rel...,cs.LG,[cs.LG],yes
4,0704.1394,Calculating Valid Domains for BDD-Based Intera...,In these notes we formally describe the functi...,cs.AI,[cs.AI],yes


## Prepare the text for vectorizing

Here we will append the title to the text.

In [26]:
df_filtered['prepared_text'] = df_filtered['title'] + ' {title} ' + df_filtered['abstract']

df_filtered.head()

,id,title,abstract,categories,code_list,selected_cats,prepared_text
0,0704.1020,The on-line shortest path problem under partia...,The on-line shortest path problem is considere...,cs.LG cs.SC,"[cs.LG, cs.SC]",yes,The on-line shortest path problem under partia...
1,0704.1028,A neural network approach to ordinal regression,Ordinal regression is an important type of lea...,cs.LG cs.AI cs.NE,"[cs.LG, cs.AI, cs.NE]",yes,A neural network approach to ordinal regressio...
2,0704.1267,Text Line Segmentation of Historical Documents...,There is a huge amount of historical documents...,cs.CV,[cs.CV],yes,Text Line Segmentation of Historical Documents...
3,0704.1274,Parametric Learning and Monte Carlo Optimization,This paper uncovers and explores the close rel...,cs.LG,[cs.LG],yes,Parametric Learning and Monte Carlo Optimizati...
4,0704.1394,Calculating Valid Domains for BDD-Based Intera...,In these notes we formally describe the functi...,cs.AI,[cs.AI],yes,Calculating Valid Domains for BDD-Based Intera...


In [19]:
df_filtered.loc[0, 'prepared_text']

'The on-line shortest path problem under partial monitoring {title} The on-line shortest path problem is considered under various models of partial monitoring. Given a weighted directed acyclic graph whose edge weights can change in an arbitrary (adversarial) way, a decision maker has to choose in each round of a game a path between two distinguished vertices such that the loss of the chosen path (defined as the sum of the weights of its composing edges) be as small as possible. In a setting generalizing the multi-armed bandit problem, after choosing a path, the decision maker learns only the weights of those edges that belong to the chosen path. For this problem, an algorithm is given whose average cumulative loss in n rounds exceeds that of the best path, matched off-line to the entire sequence of the edge weights, by a quantity that is proportional to 1/\\sqrt{n} and depends only polynomially on the number of edges of the graph. The algorithm can be implemented with linear complexit

In [27]:
# Take a small sample of the data
# This reduces the cost of vectorizing

df_sample = df_filtered[0:SAMPLE_SIZE]

df_sample = df_sample.reset_index(drop=True)

print(df_sample.shape)

df_sample.head()

(259463, 7)


,id,title,abstract,categories,code_list,selected_cats,prepared_text
0,0704.1020,The on-line shortest path problem under partia...,The on-line shortest path problem is considere...,cs.LG cs.SC,"[cs.LG, cs.SC]",yes,The on-line shortest path problem under partia...
1,0704.1028,A neural network approach to ordinal regression,Ordinal regression is an important type of lea...,cs.LG cs.AI cs.NE,"[cs.LG, cs.AI, cs.NE]",yes,A neural network approach to ordinal regressio...
2,0704.1267,Text Line Segmentation of Historical Documents...,There is a huge amount of historical documents...,cs.CV,[cs.CV],yes,Text Line Segmentation of Historical Documents...
3,0704.1274,Parametric Learning and Monte Carlo Optimization,This paper uncovers and explores the close rel...,cs.LG,[cs.LG],yes,Parametric Learning and Monte Carlo Optimizati...
4,0704.1394,Calculating Valid Domains for BDD-Based Intera...,In these notes we formally describe the functi...,cs.AI,[cs.AI],yes,Calculating Valid Domains for BDD-Based Intera...


## Get the total number of tokens

In [28]:
# Ref: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

import tiktoken

# Function to calculate the number of tokens in a string
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [29]:
# Example
num_tokens_from_string("tiktoken is great!", "cl100k_base")

6

In [30]:
def get_num_tokens(x):
    
    num_tokens = num_tokens_from_string(x, "cl100k_base")
    
    return num_tokens

df_sample['num_tokens'] = df_sample['prepared_text'].apply(get_num_tokens)

print(df_sample.shape)
print(df_sample['num_tokens'].sum())

df_sample.head()

(259463, 8)
60584665


,id,title,abstract,categories,code_list,selected_cats,prepared_text,num_tokens
0,0704.1020,The on-line shortest path problem under partia...,The on-line shortest path problem is considere...,cs.LG cs.SC,"[cs.LG, cs.SC]",yes,The on-line shortest path problem under partia...,338
1,0704.1028,A neural network approach to ordinal regression,Ordinal regression is an important type of lea...,cs.LG cs.AI cs.NE,"[cs.LG, cs.AI, cs.NE]",yes,A neural network approach to ordinal regressio...,169
2,0704.1267,Text Line Segmentation of Historical Documents...,There is a huge amount of historical documents...,cs.CV,[cs.CV],yes,Text Line Segmentation of Historical Documents...,149
3,0704.1274,Parametric Learning and Monte Carlo Optimization,This paper uncovers and explores the close rel...,cs.LG,[cs.LG],yes,Parametric Learning and Monte Carlo Optimizati...,261
4,0704.1394,Calculating Valid Domains for BDD-Based Intera...,In these notes we formally describe the functi...,cs.AI,[cs.AI],yes,Calculating Valid Domains for BDD-Based Intera...,53


## Estimate the cost for creating the embeddings

In [33]:
# Ref: https://openai.com/pricing
# ada v2	$0.00010 / 1K tokens # US Dollars

total_tokens = df_sample['num_tokens'].sum()
cost_per_1000_tokens = 0.00010 

total_cost = (total_tokens/1000) * cost_per_1000_tokens

print('Num papers:',len(df_sample))
print('Total tokens:',total_tokens)
print('Total cost (USD):',total_cost) # US Dollars

Num papers: 259463
Total tokens: 60584665
Total cost (USD): 6.058466500000001


In [51]:
qqq

NameError: name 'qqq' is not defined

## Create a list of dicts

In [52]:
# Example data format for upload to the vector database

data = [
   {
      "title": "Object0",
      "foo": 99, 
      "quote_text": "The quick brown fox jumps over the lazy dog."
   },
   {
      "title": "Object1",
      "foo": 77, 
      "quote_text": "A nimble red fox leaped over the sleeping hound."
   }
]
   

In [53]:
data_list = []

df_final = df_sample.copy()

for i in range(0, len(df_final)):
    
    data = {
        "arxiv_id": df_final.loc[i, 'id'],
        "title": df_final.loc[i, 'title'],
        "abstract": df_final.loc[i, 'prepared_text']
    }
    
    data_list.append(data)
    
len(data_list)

500

In [54]:
data_list[3]

{'arxiv_id': '0704.1394',
 'title': 'Calculating Valid Domains for BDD-Based Interactive Configuration',
 'abstract': 'Calculating Valid Domains for BDD-Based Interactive Configuration {title} In these notes we formally describe the functionality of Calculating Valid Domains from the BDD representing the solution space of valid configurations. The formalization is largely based on the CLab configuration framework.'}

## Create the vector database

In [55]:
# Connect

import weaviate
import json

client = weaviate.Client(
    url = WCS_ENDPOINT,  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WCS_API_KEY),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY  # Replace with your inference API key
    }
)

client.is_ready()

/opt/conda/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.4.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


True

In [56]:
# Configure the database

# resetting the schema. CAUTION: This will delete your collection 
if client.schema.exists(VECTOR_DB_NAME):
     client.schema.delete_class(VECTOR_DB_NAME)


# Create a Table in the database.
# We call this table a "class".

class_obj = {
    "class": VECTOR_DB_NAME, # This is the name of the database table
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-openai": {},
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)

## Upload the data to the vector database

In [58]:
BATCH_SIZE

100

In [57]:
import requests
import json

client.batch.configure(batch_size=BATCH_SIZE)  # Configure batch
with client.batch as batch:  # Initialize a batch process
    for i, data in enumerate(data_list):  # Batch import data
        print(f"importing chunk: {i+1}")
        properties = {
            "arxiv_id": data["arxiv_id"],
            "title": data["title"],
            "abstract": data["abstract"] 
        }
        batch.add_data_object(
            data_object=properties,
            class_name=VECTOR_DB_NAME
        )

importing chunk: 1
importing chunk: 2
importing chunk: 3
importing chunk: 4
importing chunk: 5
importing chunk: 6
importing chunk: 7
importing chunk: 8
importing chunk: 9
importing chunk: 10
importing chunk: 11
importing chunk: 12
importing chunk: 13
importing chunk: 14
importing chunk: 15
importing chunk: 16
importing chunk: 17
importing chunk: 18
importing chunk: 19
importing chunk: 20
importing chunk: 21
importing chunk: 22
importing chunk: 23
importing chunk: 24
importing chunk: 25
importing chunk: 26
importing chunk: 27
importing chunk: 28
importing chunk: 29
importing chunk: 30
importing chunk: 31
importing chunk: 32
importing chunk: 33
importing chunk: 34
importing chunk: 35
importing chunk: 36
importing chunk: 37
importing chunk: 38
importing chunk: 39
importing chunk: 40
importing chunk: 41
importing chunk: 42
importing chunk: 43
importing chunk: 44
importing chunk: 45
importing chunk: 46
importing chunk: 47
importing chunk: 48
importing chunk: 49
importing chunk: 50
importing

In [59]:
# Check number of objects in MyTable1

response = (
    client.query
    .aggregate(VECTOR_DB_NAME)
    .with_meta_count()
    .do()
)

print(response)

{'data': {'Aggregate': {'ARXIV_100SAMPLE_VDB': [{'meta': {'count': 500}}]}}}


## Example: Query the database

Here we will run a hybrid search.

In [60]:
# Connect to the database

import weaviate
import json

wcs_client = weaviate.Client(
    url = WCS_ENDPOINT,  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WCS_API_KEY),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY, 
        "X-Cohere-Api-Key": COHERE_API_KEY
    }
)

# check that the client is ready
wcs_client.is_ready()

True

In [61]:
#query_text = "General purpose computers"
query_text = "Quantum computing"

response = (
    client.query
    .get(VECTOR_DB_NAME, ["arxiv_id", "abstract", "title"])
    .with_near_text({"concepts": [query_text]})
    .with_limit(5)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"])
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "ARXIV_100SAMPLE_VDB": [
                {
                    "_additional": {
                        "distance": 0.1498617,
                        "id": "7d47e7df-744e-42e0-a70c-10fb1fdce2fe"
                    },
                    "abstract": "The Road to Quantum Artificial Intelligence {title} This paper overviews the basic principles and recent advances in the emerging field of Quantum Computation (QC), highlighting its potential application to Artificial Intelligence (AI). The paper provides a very brief introduction to basic QC issues like quantum registers, quantum gates and quantum algorithms and then it presents references, ideas and research guidelines on how QC can be used to deal with some basic AI problems, such as search and pattern matching, as soon as quantum computers become widely available.",
                    "arxiv_id": "0705.3360",
                    "title": "The Road to Quantum Artificial Intelligence"
      

In [62]:
# Run a hybrid search

response = (
    wcs_client.query
    .get(VECTOR_DB_NAME, ["arxiv_id", "title", "abstract"])
    .with_hybrid(query=query_text, alpha=0.5)
    .with_limit(10)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"]) # This id is the Weaviate database id
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "ARXIV_100SAMPLE_VDB": [
                {
                    "_additional": {
                        "distance": null,
                        "id": "7d47e7df-744e-42e0-a70c-10fb1fdce2fe"
                    },
                    "abstract": "The Road to Quantum Artificial Intelligence {title} This paper overviews the basic principles and recent advances in the emerging field of Quantum Computation (QC), highlighting its potential application to Artificial Intelligence (AI). The paper provides a very brief introduction to basic QC issues like quantum registers, quantum gates and quantum algorithms and then it presents references, ideas and research guidelines on how QC can be used to deal with some basic AI problems, such as search and pattern matching, as soon as quantum computers become widely available.",
                    "arxiv_id": "0705.3360",
                    "title": "The Road to Quantum Artificial Intelligence"
           

In [63]:
# Include reranking after the hybrid search

response = (
    wcs_client.query
    .get(VECTOR_DB_NAME, ["arxiv_id", "title", "abstract"])
    .with_hybrid(query=query_text, alpha=0.5)
    .with_additional(f"rerank(property: \"abstract\" query: \"{query_text}\") {{ score }}")
    .with_limit(10)
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "ARXIV_100SAMPLE_VDB": [
                {
                    "_additional": {
                        "rerank": [
                            {
                                "score": 0.97090924
                            }
                        ]
                    },
                    "abstract": "The Road to Quantum Artificial Intelligence {title} This paper overviews the basic principles and recent advances in the emerging field of Quantum Computation (QC), highlighting its potential application to Artificial Intelligence (AI). The paper provides a very brief introduction to basic QC issues like quantum registers, quantum gates and quantum algorithms and then it presents references, ideas and research guidelines on how QC can be used to deal with some basic AI problems, such as search and pattern matching, as soon as quantum computers become widely available.",
                    "arxiv_id": "0705.3360",
                    "ti